In [1]:
from flask import Flask, render_template, request, jsonify
import serial
import serial.tools.list_ports
import threading
import sys
import time
import json
import logging
from dataclasses import dataclass, field
from typing import Optional, List, Tuple
from datetime import datetime
from serial.tools import list_ports
import matplotlib.pyplot as plt
import re

Debug = 1

In [2]:
import sqlite3
from datetime import datetime, timezone, timedelta

def get_gmt7_timestamp():
    """Lấy timestamp hiện tại theo múi giờ GMT+7"""
    gmt7 = timezone(timedelta(hours=7))
    return datetime.now(gmt7)

def create_database():
    conn = sqlite3.connect('zigbee_packets.db')
    cursor = conn.cursor()
    
    # Tạo bảng với cột port mới
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS zigbee_packets (
            ID TEXT,
            event TEXT,
            data TEXT,
            time INTEGER,
            rssi INTEGER,
            antena_id INTEGER,
            sync_word INTEGER,
            is_ack INTEGER,
            crc_pass INTEGER,
            lqi INTEGER,
            channel INTEGER,
            created_at TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    print("✅ Database và table đã được tạo thành công")

def create_escan_database():
    """Tạo database đơn giản cho EScan"""
    conn = sqlite3.connect('escan_results.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS escan_data (
            ID64 TEXT NOT NULL,
            channel INTEGER NOT NULL,
            energy_value REAL NOT NULL,
            scan_timestamp TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    print("✅ Database escan_results.db đã được tạo thành công")

def save_escan_result(node_port, channel, energy_value):
    """Lưu kết quả EScan vào database - đơn giản"""
    try:
        conn = sqlite3.connect('escan_results.db')
        cursor = conn.cursor()
        
        # Tạo timestamp GMT+7
        gmt7_time = get_gmt7_timestamp()
        
        cursor.execute('''
            INSERT INTO escan_data (ID64, channel, energy_value, scan_timestamp)
            VALUES (?, ?, ?, ?)
        ''', (node_port, channel, energy_value, gmt7_time))
        
        conn.commit()
        conn.close()
        
        try:
            if Debug >= 1:
                print(f"✅ Đã lưu EScan: ID={node_port}, Ch={channel}, Energy={energy_value}")
        except NameError:
            # Nếu Debug chưa được định nghĩa, không in gì cả
            pass
        
    except Exception as e:
        print(f"❌ Lỗi khi lưu EScan: {e}")

def get_node_id64_from_packets(Self):
    """Lấy ID64 từ packet gần nhất của node"""
    if Self.Packets and len(Self.Packets) > 0:
        for packet in reversed(Self.Packets):  # Duyệt từ packet mới nhất
            if hasattr(packet, 'ID64') and packet.ID64 and packet.ID64 != "":
                return packet.ID64
    
    # Fallback: tạo ID64 từ port
    return f"NODE_{Self.Port.replace('COM', '').replace('/', '_')}"


In [ ]:
@dataclass
class DataEntry:
    TimeSend: str
    TimeRead: str
    Cmd: str
    Responses: List[str] = field(default_factory=list)

@dataclass
class IEEE802154FrameControlAll:
    FrameControl: str = ""
    FrameType: str = ""
    SecurityEn: str = ""
    FramePending: str = ""
    AckRequied: str = ""
    PanIDCompression: str = ""
    FrameVersion: str = ""
    Reserved: str = ""
    DestinationAddrMode: str = ""
    SouceAddrMode: str = ""

@dataclass
class ZigbeeNetworkFrameControlAll:
    FrameControl: str = ""
    FrameType: str = ""
    ProtocoVersion: str = ""
    DiscoverRouter: str = ""
    Multicast: str = ""
    Security: str = ""
    SouceRouter: str = ""
    LongDestPresent: str = ""
    LongSoucePresent: str = ""
    EndDeviceInitiator: str = ""

@dataclass
class ZigbeeNetworkSecurityFrameControlAll:
    FrameControl: str = ""
    SecurityLevel: str = ""
    KeyIdentifer: str = ""
    ExtenedNonce: str = ""

@dataclass
class IEEE802154:
    PHYHeader: str = ""
    Sequence: str = ""
    DestinationPANID: str = ""
    ShortDestinationAddr: str = ""
    ShortSouceAddr: str = ""
    FrameControl: IEEE802154FrameControlAll = field(default_factory=IEEE802154FrameControlAll)

@dataclass
class ZigbeeNetwork:
    DestinationAddr: str = ""
    SouceAddr: str = ""
    Radius: str = ""
    Sequence: str = ""
    FrameControl: ZigbeeNetworkFrameControlAll = field(default_factory=ZigbeeNetworkFrameControlAll)

@dataclass
class ZigbeeNetworkSecurity:
    FrameCouter: str = ""
    SouceAddr: str = ""
    KeySequence: str = ""
    FrameControl: ZigbeeNetworkSecurityFrameControlAll = field(default_factory=ZigbeeNetworkSecurityFrameControlAll)

@dataclass
class ZigbeePacket:
    Event: str = ""
    Data: str = ""
    ID64: str = ""
    Time: int = 0
    RSSI: int = 0
    AntenaID: int = 0
    SyncWord: int = 0
    IsACK: int = 0
    CRCPass: int = 0
    LQI: int = 0
    Channel: int = 0
    TypePackage: str = ""
    Interface: IEEE802154 = field(default_factory=IEEE802154)
    ZigbeeNet: ZigbeeNetwork = field(default_factory=ZigbeeNetwork)
    ZigbeeNetSec: ZigbeeNetworkSecurity = field(default_factory=ZigbeeNetworkSecurity)
    ApplicationPayload: List[str] = field(default_factory=list)
    DataBytes: List[str] = field(default_factory=list)

    def Decode(Self, DataBytes: List[str]):
        if not DataBytes or len(DataBytes) < 3:
            Self.TypePackage = "Invalid"
            return False
        
        try:
            # Parse IEEE 802.15.4 Layer
            Self.Interface.PHYHeader = DataBytes[0]
            
            FCFLow = int(DataBytes[1], 16)
            FCFHigh = int(DataBytes[2], 16)
            FCFValue = (FCFHigh << 8) | FCFLow
            
            FrameType = FCFValue & 0x07
            SecurityEnabled = bool((FCFValue >> 3) & 0x01)
            FramePending = bool((FCFValue >> 4) & 0x01)
            ACKRequest = bool((FCFValue >> 5) & 0x01)
            PANIDCompression = bool((FCFValue >> 6) & 0x01)
            Reserved = (FCFValue >> 7) & 0x07
            DestAddrMode = (FCFValue >> 10) & 0x03
            FrameVersion = (FCFValue >> 12) & 0x03
            SrcAddrMode = (FCFValue >> 14) & 0x03
            
            FrameTypeMap = {
                0b000: "Beacon",
                0b001: "Data", 
                0b010: "ACK",
                0b011: "Command"
            }
            
            AddrModeMap = {0: "None", 1: "Reserved", 2: "Short", 3: "Extended"}
            
            FrameTypeName = FrameTypeMap.get(FrameType, f"Reserved({FrameType})")
            
            # Fill IEEE 802.15.4 Frame Control - ĐẦY ĐỦ TẤT CẢ TRƯỜNG
            Self.Interface.FrameControl.FrameControl = f"{FCFValue:04X}"
            Self.Interface.FrameControl.FrameType = FrameTypeName
            Self.Interface.FrameControl.SecurityEn = "True" if SecurityEnabled else "False"
            Self.Interface.FrameControl.FramePending = "True" if FramePending else "False"
            Self.Interface.FrameControl.AckRequied = "True" if ACKRequest else "False"
            Self.Interface.FrameControl.PanIDCompression = "True" if PANIDCompression else "False"
            Self.Interface.FrameControl.FrameVersion = str(FrameVersion)
            Self.Interface.FrameControl.Reserved = f"{Reserved:03b}"
            Self.Interface.FrameControl.DestinationAddrMode = AddrModeMap.get(DestAddrMode, "Unknown")
            Self.Interface.FrameControl.SouceAddrMode = AddrModeMap.get(SrcAddrMode, "Unknown")
            
            ByteIndex = 3
            
            # Parse addressing based on frame type
            if FrameTypeName == "ACK":
                if len(DataBytes) > ByteIndex:
                    Self.Interface.Sequence = DataBytes[ByteIndex]
                Self.TypePackage = "ACK"
                # RESET CÁC TRƯỜNG KHÔNG SỬ DỤNG
                Self.Interface.DestinationPANID = ""
                Self.Interface.ShortDestinationAddr = ""
                Self.Interface.ShortSouceAddr = ""
                return True
                
            elif FrameTypeName == "Beacon":
                if len(DataBytes) > ByteIndex:
                    Self.Interface.Sequence = DataBytes[ByteIndex]
                    ByteIndex += 1
                
                if SrcAddrMode != 0:
                    if len(DataBytes) > ByteIndex + 1:
                        Self.Interface.DestinationPANID = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                        ByteIndex += 2
                        
                    if SrcAddrMode == 2:
                        if len(DataBytes) > ByteIndex + 1:
                            Self.Interface.ShortSouceAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                            ByteIndex += 2
                    elif SrcAddrMode == 3:
                        ByteIndex += 8  # THÊM: Extended source address
                            
                Self.TypePackage = "Beacon"
                Self.Interface.ShortDestinationAddr = ""
                return True
                
            elif FrameTypeName == "Command":
                if len(DataBytes) > ByteIndex:
                    Self.Interface.Sequence = DataBytes[ByteIndex]
                    ByteIndex += 1
                    
                # Parse addressing for Command frames
                if DestAddrMode != 0:
                    if len(DataBytes) > ByteIndex + 1:
                        Self.Interface.DestinationPANID = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                        ByteIndex += 2
                        
                        if DestAddrMode == 2:
                            if len(DataBytes) > ByteIndex + 1:
                                Self.Interface.ShortDestinationAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                                ByteIndex += 2
                        elif DestAddrMode == 3:
                            ByteIndex += 8  # Extended destination address
                
                if SrcAddrMode != 0 and not PANIDCompression:
                    ByteIndex += 2  # Source PAN ID
                    
                if SrcAddrMode == 2:
                    if len(DataBytes) > ByteIndex + 1:
                        Self.Interface.ShortSouceAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                        ByteIndex += 2
                elif SrcAddrMode == 3:
                    ByteIndex += 8  # Extended source address
                
                # Determine specific command type
                if len(DataBytes) > ByteIndex:
                    CommandID = int(DataBytes[ByteIndex], 16)
                    CommandMap = {
                        0x01: "Association Request",
                        0x02: "Association Response", 
                        0x04: "Data Request",
                        0x07: "Beacon Request"
                    }
                    Self.TypePackage = CommandMap.get(CommandID, f"Command({CommandID:02X})")
                    ByteIndex += 1  # THÊM: Tăng ByteIndex sau khi đọc Command ID
                else:
                    Self.TypePackage = "Command"
                return True
                
            elif FrameTypeName == "Data":
                if len(DataBytes) > ByteIndex:
                    Self.Interface.Sequence = DataBytes[ByteIndex]
                    ByteIndex += 1
                    
                # Parse addressing for Data frames
                if DestAddrMode != 0:
                    if len(DataBytes) > ByteIndex + 1:
                        Self.Interface.DestinationPANID = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                        ByteIndex += 2
                        
                        if DestAddrMode == 2:
                            if len(DataBytes) > ByteIndex + 1:
                                Self.Interface.ShortDestinationAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                                ByteIndex += 2
                        elif DestAddrMode == 3:
                            ByteIndex += 8  # Extended destination address
                
                if SrcAddrMode != 0 and not PANIDCompression:
                    ByteIndex += 2  # Source PAN ID
                    
                if SrcAddrMode == 2:
                    if len(DataBytes) > ByteIndex + 1:
                        Self.Interface.ShortSouceAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                        ByteIndex += 2
                elif SrcAddrMode == 3:
                    ByteIndex += 8  # Extended source address
                
                # Parse ZigBee Network Layer if present
                if len(DataBytes) > ByteIndex + 1:
                    ZigbeePayloadStart = ByteIndex
                    if Self.ParseZigbeeNetwork(DataBytes, ZigbeePayloadStart):
                        Self.TypePackage = "ZigBee Data"
                    else:
                        Self.TypePackage = "Data"
                else:
                    Self.TypePackage = "Data"
                return True
                
            else:
                Self.TypePackage = f"Reserved({FrameType})"
                return True
                
        except (ValueError, IndexError) as e:
            Self.TypePackage = "Parse Error"
            return False

    def ParseZigbeeNetwork(Self, DataBytes: List[str], StartIndex: int):
        try:
            if len(DataBytes) <= StartIndex + 1:
                return False
                
            # Parse ZigBee Network Frame Control
            NwkFCFLow = int(DataBytes[StartIndex], 16)
            NwkFCFHigh = int(DataBytes[StartIndex + 1], 16)
            NwkFCFValue = (NwkFCFHigh << 8) | NwkFCFLow
            
            NwkFrameType = NwkFCFValue & 0x03
            ProtocolVersion = (NwkFCFValue >> 2) & 0x0F
            DiscoverRoute = (NwkFCFValue >> 6) & 0x03  # SỬA: 2 bits thay vì 1 bit
            Multicast = bool((NwkFCFValue >> 8) & 0x01)
            Security = bool((NwkFCFValue >> 9) & 0x01)
            SourceRoute = bool((NwkFCFValue >> 10) & 0x01)
            DestExtended = bool((NwkFCFValue >> 11) & 0x01)
            SourceExtended = bool((NwkFCFValue >> 12) & 0x01)
            EndDeviceInitiator = bool((NwkFCFValue >> 13) & 0x01)
            # THÊM: Bits 14-15 reserved
            Reserved = (NwkFCFValue >> 14) & 0x03
            
            NwkFrameTypeMap = {
                0x00: "Data",
                0x01: "Command"
            }
            
            # Fill ZigBee Network Frame Control - ĐẦY ĐỦ TẤT CẢ TRƯỜNG
            Self.ZigbeeNet.FrameControl.FrameControl = f"{NwkFCFValue:04X}"
            Self.ZigbeeNet.FrameControl.FrameType = NwkFrameTypeMap.get(NwkFrameType, f"Reserved({NwkFrameType})")
            Self.ZigbeeNet.FrameControl.ProtocoVersion = str(ProtocolVersion)
            Self.ZigbeeNet.FrameControl.DiscoverRouter = str(DiscoverRoute)  # SỬA: Lưu giá trị số thay vì boolean
            Self.ZigbeeNet.FrameControl.Multicast = "True" if Multicast else "False"
            Self.ZigbeeNet.FrameControl.Security = "True" if Security else "False"
            Self.ZigbeeNet.FrameControl.SouceRouter = "True" if SourceRoute else "False"
            Self.ZigbeeNet.FrameControl.LongDestPresent = "True" if DestExtended else "False"
            Self.ZigbeeNet.FrameControl.LongSoucePresent = "True" if SourceExtended else "False"
            Self.ZigbeeNet.FrameControl.EndDeviceInitiator = "True" if EndDeviceInitiator else "False"
            
            ByteIndex = StartIndex + 2
            
            # Parse destination address
            if len(DataBytes) > ByteIndex + 1:
                Self.ZigbeeNet.DestinationAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                ByteIndex += 2
            
            # Parse source address  
            if len(DataBytes) > ByteIndex + 1:
                Self.ZigbeeNet.SouceAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                ByteIndex += 2
            
            # Parse radius
            if len(DataBytes) > ByteIndex:
                Self.ZigbeeNet.Radius = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # Parse sequence
            if len(DataBytes) > ByteIndex:
                Self.ZigbeeNet.Sequence = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # THÊM: Parse extended destination address if present
            if DestExtended and len(DataBytes) > ByteIndex + 7:
                ExtendedDestAddr = ""
                for i in range(8):
                    ExtendedDestAddr = DataBytes[ByteIndex + i] + ExtendedDestAddr
                # Có thể lưu vào trường mở rộng nếu cần
                ByteIndex += 8
                
            # THÊM: Parse extended source address if present
            if SourceExtended and len(DataBytes) > ByteIndex + 7:
                ExtendedSrcAddr = ""
                for i in range(8):
                    ExtendedSrcAddr = DataBytes[ByteIndex + i] + ExtendedSrcAddr
                # Có thể lưu vào trường mở rộng nếu cần
                ByteIndex += 8
            
            # THÊM: Parse multicast control if multicast
            if Multicast and len(DataBytes) > ByteIndex:
                MulticastControl = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # THÊM: Parse source route if present
            if SourceRoute and len(DataBytes) > ByteIndex:
                RelayCount = int(DataBytes[ByteIndex], 16)
                ByteIndex += 1
                RelayIndex = int(DataBytes[ByteIndex], 16) if len(DataBytes) > ByteIndex else 0
                ByteIndex += 1
                # Skip relay list
                ByteIndex += RelayCount * 2
            
            # Parse ZigBee Security if enabled
            if Security and len(DataBytes) > ByteIndex:
                Self.ParseZigbeeNetworkSecurity(DataBytes, ByteIndex)
            else:
                # Remaining bytes are application payload
                if len(DataBytes) > ByteIndex:
                    Self.ApplicationPayload = DataBytes[ByteIndex:]
            
            return True
            
        except (ValueError, IndexError):
            return False

    def ParseZigbeeNetworkSecurity(Self, DataBytes: List[str], StartIndex: int):
        try:
            if len(DataBytes) <= StartIndex:
                return False
                
            # Parse Security Control Field
            SecCtrl = int(DataBytes[StartIndex], 16)
            SecurityLevel = SecCtrl & 0x07
            KeyIdentifier = (SecCtrl >> 3) & 0x03
            ExtendedNonce = bool((SecCtrl >> 5) & 0x01)
            # THÊM: Reserved bits
            Reserved = (SecCtrl >> 6) & 0x03
            
            # Fill ZigBee Network Security Frame Control - ĐẦY ĐỦ TẤT CẢ TRƯỜNG
            Self.ZigbeeNetSec.FrameControl.FrameControl = f"{SecCtrl:02X}"
            Self.ZigbeeNetSec.FrameControl.SecurityLevel = str(SecurityLevel)
            Self.ZigbeeNetSec.FrameControl.KeyIdentifer = str(KeyIdentifier)
            Self.ZigbeeNetSec.FrameControl.ExtenedNonce = "True" if ExtendedNonce else "False"
            
            ByteIndex = StartIndex + 1
            
            # Parse Frame Counter (4 bytes, Little Endian)
            if len(DataBytes) > ByteIndex + 3:
                FrameCounter = ""
                for i in range(4):
                    FrameCounter = DataBytes[ByteIndex + i] + FrameCounter
                Self.ZigbeeNetSec.FrameCouter = FrameCounter
                ByteIndex += 4
            
            # Parse Source Address if Extended Nonce
            if ExtendedNonce and len(DataBytes) > ByteIndex + 7:
                SourceAddr = ""
                for i in range(8):
                    SourceAddr = DataBytes[ByteIndex + i] + SourceAddr
                Self.ZigbeeNetSec.SouceAddr = SourceAddr
                ByteIndex += 8
            
            # Parse Key Sequence Number if needed
            if KeyIdentifier == 1 and len(DataBytes) > ByteIndex:
                Self.ZigbeeNetSec.KeySequence = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # THÊM: Parse MIC (Message Integrity Code) dựa trên Security Level
            MICLength = 0
            if SecurityLevel in [1, 5]:
                MICLength = 4
            elif SecurityLevel in [2, 6]:
                MICLength = 8
            elif SecurityLevel in [3, 7]:
                MICLength = 16
            
            if MICLength > 0 and len(DataBytes) >= ByteIndex + MICLength:
                # MIC ở cuối frame, lùi từ cuối
                PayloadEnd = len(DataBytes) - MICLength
                if PayloadEnd > ByteIndex:
                    Self.ApplicationPayload = DataBytes[ByteIndex:PayloadEnd]
                # MIC = DataBytes[PayloadEnd:] nếu cần lưu
            else:
                # Remaining bytes are encrypted application payload
                if len(DataBytes) > ByteIndex:
                    Self.ApplicationPayload = DataBytes[ByteIndex:]
            
            return True
            
        except (ValueError, IndexError):
            return False

    def ParseZigbeeNetwork(Self, DataBytes: List[str], StartIndex: int):
        try:
            if len(DataBytes) <= StartIndex + 1:
                return False
                
            # Parse ZigBee Network Frame Control
            NwkFCFLow = int(DataBytes[StartIndex], 16)
            NwkFCFHigh = int(DataBytes[StartIndex + 1], 16)
            NwkFCFValue = (NwkFCFHigh << 8) | NwkFCFLow
            
            NwkFrameType = NwkFCFValue & 0x03
            ProtocolVersion = (NwkFCFValue >> 2) & 0x0F
            DiscoverRoute = bool((NwkFCFValue >> 6) & 0x03)
            Multicast = bool((NwkFCFValue >> 8) & 0x01)
            Security = bool((NwkFCFValue >> 9) & 0x01)
            SourceRoute = bool((NwkFCFValue >> 10) & 0x01)
            DestExtended = bool((NwkFCFValue >> 11) & 0x01)
            SourceExtended = bool((NwkFCFValue >> 12) & 0x01)
            EndDeviceInitiator = bool((NwkFCFValue >> 13) & 0x01)
            
            NwkFrameTypeMap = {
                0x00: "Data",
                0x01: "Command"
            }
            
            # Fill ZigBee Network Frame Control
            Self.ZigbeeNet.FrameControl.FrameControl = f"{NwkFCFValue:04X}"
            Self.ZigbeeNet.FrameControl.FrameType = NwkFrameTypeMap.get(NwkFrameType, f"Reserved({NwkFrameType})")
            Self.ZigbeeNet.FrameControl.ProtocoVersion = str(ProtocolVersion)
            Self.ZigbeeNet.FrameControl.DiscoverRouter = "True" if DiscoverRoute else "False"
            Self.ZigbeeNet.FrameControl.Multicast = "True" if Multicast else "False"
            Self.ZigbeeNet.FrameControl.Security = "True" if Security else "False"
            Self.ZigbeeNet.FrameControl.SouceRouter = "True" if SourceRoute else "False"
            Self.ZigbeeNet.FrameControl.LongDestPresent = "True" if DestExtended else "False"
            Self.ZigbeeNet.FrameControl.LongSoucePresent = "True" if SourceExtended else "False"
            Self.ZigbeeNet.FrameControl.EndDeviceInitiator = "True" if EndDeviceInitiator else "False"
            
            ByteIndex = StartIndex + 2
            
            # Parse destination address
            if len(DataBytes) > ByteIndex + 1:
                Self.ZigbeeNet.DestinationAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                ByteIndex += 2
            
            # Parse source address  
            if len(DataBytes) > ByteIndex + 1:
                Self.ZigbeeNet.SouceAddr = f"{DataBytes[ByteIndex+1]}{DataBytes[ByteIndex]}"
                ByteIndex += 2
            
            # Parse radius
            if len(DataBytes) > ByteIndex:
                Self.ZigbeeNet.Radius = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # Parse sequence
            if len(DataBytes) > ByteIndex:
                Self.ZigbeeNet.Sequence = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # Parse extended addresses if present
            if DestExtended and len(DataBytes) > ByteIndex + 7:
                ByteIndex += 8
                
            if SourceExtended and len(DataBytes) > ByteIndex + 7:
                ByteIndex += 8
            
            # Parse ZigBee Security if enabled
            if Security and len(DataBytes) > ByteIndex:
                Self.ParseZigbeeNetworkSecurity(DataBytes, ByteIndex)
            else:
                # Remaining bytes are application payload
                if len(DataBytes) > ByteIndex:
                    Self.ApplicationPayload = DataBytes[ByteIndex:]
            
            return True
            
        except (ValueError, IndexError):
            return False

    def ParseZigbeeNetworkSecurity(Self, DataBytes: List[str], StartIndex: int):
        try:
            if len(DataBytes) <= StartIndex:
                return False
                
            # Parse Security Control Field
            SecCtrl = int(DataBytes[StartIndex], 16)
            SecurityLevel = SecCtrl & 0x07
            KeyIdentifier = (SecCtrl >> 3) & 0x03
            ExtendedNonce = bool((SecCtrl >> 5) & 0x01)
            
            # Fill ZigBee Network Security Frame Control
            Self.ZigbeeNetSec.FrameControl.FrameControl = f"{SecCtrl:02X}"
            Self.ZigbeeNetSec.FrameControl.SecurityLevel = str(SecurityLevel)
            Self.ZigbeeNetSec.FrameControl.KeyIdentifer = str(KeyIdentifier)
            Self.ZigbeeNetSec.FrameControl.ExtenedNonce = "True" if ExtendedNonce else "False"
            
            ByteIndex = StartIndex + 1
            
            # Parse Frame Counter (4 bytes)
            if len(DataBytes) > ByteIndex + 3:
                FrameCounter = ""
                for i in range(4):
                    FrameCounter = DataBytes[ByteIndex + i] + FrameCounter
                Self.ZigbeeNetSec.FrameCouter = FrameCounter
                ByteIndex += 4
            
            # Parse Source Address if Extended Nonce
            if ExtendedNonce and len(DataBytes) > ByteIndex + 7:
                SourceAddr = ""
                for i in range(8):
                    SourceAddr = DataBytes[ByteIndex + i] + SourceAddr
                Self.ZigbeeNetSec.SouceAddr = SourceAddr
                ByteIndex += 8
            
            # Parse Key Sequence Number if needed
            if KeyIdentifier == 1 and len(DataBytes) > ByteIndex:
                Self.ZigbeeNetSec.KeySequence = DataBytes[ByteIndex]
                ByteIndex += 1
            
            # Remaining bytes are encrypted application payload
            if len(DataBytes) > ByteIndex:
                Self.ApplicationPayload = DataBytes[ByteIndex:]
            
            return True
            
        except (ValueError, IndexError):
            return False
        
    def save_to_db(self):
        """Lưu ZigbeePacket vào SQLite database"""
        try:
            conn = sqlite3.connect('zigbee_packets.db')
            cursor = conn.cursor()
            gmt7_time = get_gmt7_timestamp()
          
           
        
            cursor.execute('''
                INSERT INTO zigbee_packets 
                (ID, event, data, time, rssi, antena_id, sync_word, 
                 is_ack, crc_pass, lqi, channel, created_at)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                self.ID64,
                self.Event,
                self.Data,
                self.Time,
                self.RSSI,
                self.AntenaID,
                self.SyncWord,
                self.IsACK,
                self.CRCPass,
                self.LQI,
                self.Channel,
                gmt7_time 
        
                
            ))
            
            conn.commit()
            conn.close()
            if Debug >= 1:
                print(f"✅ Đã lưu packet {self.Event} vào database")
            
        except Exception as e:
            print(f"❌ Lỗi khi lưu vào database: {e}")
    
    def ExtractData(self, String: str):
        self.Event = ""
        self.Data = ""
        self.Time = 0
        self.ID64 = ""
        self.RSSI = 0
        self.LQI = 0
        self.CRCPass = 0
        self.SyncWord = 0
        self.AntenaID = 0
        self.IsACK = 0
        self.Channel = 0
        self.SubPhyID = 0
        self.DataBytes = []
        
        
        FieldMappings = {
            'Event': (str, 'Event'),
            'Time': (int, 'Time'),
            'RSSI': (int, 'RSSI'),
            'LQI': (int, 'LQI'),
            'CRCPass': (int, 'CRCPass'),
            'SubPhyID': (int, 'SubPhyID'),
            'IsACK': (int, 'IsACK'),
            'SyncWord': (int, 'SyncWord'),  
            'AntenaID': (int, 'AntenaID'),
            'ID64': (str, 'ID64'),
            'Channel': (int, 'Channel')
        }
        
        EventMatch = re.search(r'\{Event: ([^}]+)\}', String)
        if not EventMatch:
            return False
        self.Event = EventMatch.group(1)
        
        DataMatch = re.search(r'\{Data:\{([^}]*)\}', String)
        if DataMatch:
            DataStr = DataMatch.group(1).strip()
            if DataStr:
                self.Data = DataStr
                self.DataBytes = [Byte.strip() for Byte in DataStr.split() if Byte.strip()]
        ID64Match = re.search(r'\{ID64: ([^}]+)\}', String)
        if ID64Match:
            self.ID64 = ID64Match.group(1).strip()
        for FieldName, (DataType, AttrName) in FieldMappings.items():
            if FieldName == 'Event':
                continue
            
            Pattern = rf'\{{{FieldName}: (-?\d+)\}}' if DataType == int else rf'\{{{FieldName}: ([^}}]+)\}}'
            Match = re.search(Pattern, String)
            if Match:
                try:
                    Value = DataType(Match.group(1))
                    setattr(self, AttrName, Value)
                except:
                    pass
        if self.Event:  # Chỉ lưu khi có Event
            self.save_to_db()
        return True
    
    def PrintPackage(Self):
        """In thông tin chi tiết ZigbeePacket theo phong cách Wireshark"""
        try:
            print("▼ IEEE 802.15.4 [8 bytes]")
            
            # PHY Header
            if Self.Interface and Self.Interface.PHYHeader:
                phy_length = int(Self.Interface.PHYHeader, 16) if Self.Interface.PHYHeader else 0
                print(f"    PHY Header: 0x{Self.Interface.PHYHeader}")
                print(f"        {phy_length:08b} = Packet Length: {phy_length}")
            
            # Frame Control Field
            if Self.Interface and Self.Interface.FrameControl and Self.Interface.FrameControl.FrameControl:
                fc = Self.Interface.FrameControl
                fcf_value = int(fc.FrameControl, 16) if fc.FrameControl else 0
                
                print(f"    Frame Control: 0x{fc.FrameControl}")
                
                # Frame Type
                frame_type_bits = fcf_value & 0x07
                frame_type_binary = f"{frame_type_bits:03b}"
                print(f"        .... .... .... .{frame_type_binary} = Frame Type: {fc.FrameType} ({frame_type_bits})")
                
                # Security Enabled
                security_bit = (fcf_value >> 3) & 0x01
                print(f"        .... .... .... {security_bit}... = Security Enabled: {fc.SecurityEn.lower()}")
                
                # Frame Pending
                pending_bit = (fcf_value >> 4) & 0x01
                print(f"        .... .... ...{pending_bit} .... = Frame Pending: {fc.FramePending.lower()}")
                
                # ACK Request
                ack_bit = (fcf_value >> 5) & 0x01
                print(f"        .... .... ..{ack_bit}. .... = Ack Required: {fc.AckRequied.lower()}")
                
                # PAN ID Compression
                pan_bit = (fcf_value >> 6) & 0x01
                print(f"        .... .... .{pan_bit}.. .... = PAN ID Compression: {fc.PanIDCompression.lower()}")
                
                # Reserved
                reserved_bits = (fcf_value >> 7) & 0x07
                reserved_binary = f"{reserved_bits:03b}"
                print(f"        .... .{reserved_binary} .... .... = Reserved: 0x{reserved_bits:01X}")
                
                # Destination Addressing Mode
                dest_mode = (fcf_value >> 10) & 0x03
                dest_binary = f"{dest_mode:02b}"
                print(f"        ..{dest_binary} .... .... .... = Destination Address Mode: {fc.DestinationAddrMode} ({dest_mode})")
                
                # Frame Version
                version = (fcf_value >> 12) & 0x03
                version_binary = f"{version:02b}"
                print(f"        .{version_binary}. .... .... .... = Frame Version: 802.15.4-2003 ({version})")
                
                # Source Addressing Mode
                src_mode = (fcf_value >> 14) & 0x03
                src_binary = f"{src_mode:02b}"
                print(f"        {src_binary}.. .... .... .... = Source Address Mode: {fc.SouceAddrMode} ({src_mode})")
            
            # Sequence Number
            if Self.Interface and Self.Interface.Sequence:
                print(f"    Sequence: 0x{Self.Interface.Sequence}")
            
            # Addressing Fields
            if Self.Interface:
                if Self.Interface.DestinationPANID:
                    print(f"    Source PAN ID: 0x{Self.Interface.DestinationPANID}")
                if Self.Interface.ShortSouceAddr:
                    print(f"    Short Source Address: 0x{Self.Interface.ShortSouceAddr}")
            
            # ZigBee Network Layer nếu có
            if Self.ZigbeeNet and Self.ZigbeeNet.FrameControl and Self.ZigbeeNet.FrameControl.FrameControl:
                print("▼ ZigBee Network Layer")
                nfc = Self.ZigbeeNet.FrameControl
                nwk_fcf = int(nfc.FrameControl, 16) if nfc.FrameControl else 0
                
                print(f"    Frame Control: 0x{nfc.FrameControl}")
                
                # Network Frame Type
                nwk_type = nwk_fcf & 0x03
                nwk_type_binary = f"{nwk_type:02b}"
                print(f"        .... .... .... ..{nwk_type_binary} = Frame Type: {nfc.FrameType} ({nwk_type})")
                
                # Protocol Version
                proto_version = (nwk_fcf >> 2) & 0x0F
                proto_binary = f"{proto_version:04b}"
                print(f"        .... .... {proto_binary} ..{nwk_type_binary} = Protocol Version: {nfc.ProtocoVersion}")
                
                # Discover Route
                discover = (nwk_fcf >> 6) & 0x03
                discover_binary = f"{discover:02b}"
                print(f"        .... ..{discover_binary} .... .... = Discover Route: {nfc.DiscoverRouter}")
                
                # Multicast
                multicast_bit = (nwk_fcf >> 8) & 0x01
                print(f"        .... .{multicast_bit}.. .... .... = Multicast: {nfc.Multicast.lower()}")
                
                # Security
                security_bit = (nwk_fcf >> 9) & 0x01
                print(f"        .... {security_bit}... .... .... = Security: {nfc.Security.lower()}")
                
                # Source Route
                src_route_bit = (nwk_fcf >> 10) & 0x01
                print(f"        ...{src_route_bit} .... .... .... = Source Route: {nfc.SouceRouter.lower()}")
                
                # Extended addresses
                dest_ext_bit = (nwk_fcf >> 11) & 0x01
                src_ext_bit = (nwk_fcf >> 12) & 0x01
                print(f"        ..{dest_ext_bit}. .... .... .... = Long Dest Present: {nfc.LongDestPresent.lower()}")
                print(f"        .{src_ext_bit}.. .... .... .... = Long Source Present: {nfc.LongSoucePresent.lower()}")
                
                # End Device Initiator
                edi_bit = (nwk_fcf >> 13) & 0x01
                print(f"        {edi_bit}... .... .... .... = End Device Initiator: {nfc.EndDeviceInitiator.lower()}")
                
                # Network addresses
                if Self.ZigbeeNet.DestinationAddr:
                    print(f"    Destination Address: {Self.ZigbeeNet.DestinationAddr}")
                if Self.ZigbeeNet.SouceAddr:
                    print(f"    Source Address: {Self.ZigbeeNet.SouceAddr}")
                if Self.ZigbeeNet.Radius:
                    print(f"    Radius: {Self.ZigbeeNet.Radius}")
                if Self.ZigbeeNet.Sequence:
                    print(f"    Sequence: {Self.ZigbeeNet.Sequence}")
            
            # ZigBee Security Layer nếu có
            if Self.ZigbeeNetSec and Self.ZigbeeNetSec.FrameControl and Self.ZigbeeNetSec.FrameControl.FrameControl:
                print("▼ ZigBee Network Security")
                sfc = Self.ZigbeeNetSec.FrameControl
                sec_fcf = int(sfc.FrameControl, 16) if sfc.FrameControl else 0
                
                print(f"    Frame Control: 0x{sfc.FrameControl}")
                
                # Security Level
                sec_level = sec_fcf & 0x07
                sec_level_binary = f"{sec_level:03b}"
                print(f"        .... .{sec_level_binary} = Security Level: No encryption, no MIC ({sec_level})")
                
                # Key Identifier
                key_id = (sec_fcf >> 3) & 0x03
                key_id_binary = f"{key_id:02b}"
                print(f"        .... {key_id_binary}.. = Key Identifier: {sfc.KeyIdentifer}")
                
                # Extended Nonce
                ext_nonce_bit = (sec_fcf >> 5) & 0x01
                print(f"        ...{ext_nonce_bit} .... = Extended Nonce: {sfc.ExtenedNonce.lower()}")
                
                if Self.ZigbeeNetSec.FrameCouter:
                    print(f"    Frame Counter: {Self.ZigbeeNetSec.FrameCouter}")
                if Self.ZigbeeNetSec.SouceAddr:
                    print(f"    Source Address: {Self.ZigbeeNetSec.SouceAddr}")
                if Self.ZigbeeNetSec.KeySequence:
                    print(f"    Key Sequence Number: {Self.ZigbeeNetSec.KeySequence}")
            
            # Application Payload
            if Self.ApplicationPayload:
                payload_length = len(Self.ApplicationPayload)
                print(f"▼ Application Payload [{payload_length} bytes]")
                
                # In payload theo format hex
                payload_hex = ' '.join(Self.ApplicationPayload)
                print(f"    Payload: {payload_hex}")
                
                # In theo dạng hex dump như Wireshark
                for i in range(0, len(Self.ApplicationPayload), 16):
                    line_bytes = Self.ApplicationPayload[i:i+16]
                    hex_str = ' '.join([f"{byte:>2}" for byte in line_bytes])
                    print(f"        {i:04X}: {hex_str}")
            
            # Thông tin bổ sung
            print("▼ Additional Information")
            if Self.TypePackage:
                print(f"    Packet Type: {Self.TypePackage}")
            if Self.Event:
                print(f"    Event: {Self.Event}")
            if Self.ID64:
                print(f"    ID64: {Self.ID64}")
            if Self.Time:
                print(f"    Time: {Self.Time} μs")
            if Self.RSSI:
                print(f"    RSSI: {Self.RSSI} dBm")
            if Self.LQI:
                print(f"    LQI: {Self.LQI}")
            if Self.Channel:
                print(f"    Channel: {Self.Channel}")
            
        except Exception as e:
            print(f"❌ Error printing package: {e}")
            # Fallback đơn giản
            print("▼ Basic Packet Information")
            print(f"    Event: {getattr(Self, 'Event', 'N/A')}")
            print(f"    ID64: {getattr(Self, 'ID64', 'N/A')}")
            print(f"    Time: {getattr(Self, 'Time', 0)} μs")
            print(f"    TypePackage: {getattr(Self, 'TypePackage', 'N/A')}")


    def HasData(self) -> bool:
        return len(self.DataBytes) > 0
    
    def GetDataLength(self) -> int:
        return len(self.DataBytes)
    
    def GetFormattedData(self) -> str:
        if not self.DataBytes:
            return "NULL"
        return " ".join([f"0x{Byte}" for Byte in self.DataBytes])
    
    def ToPacketTableFormat(self) -> dict:
        return {
            'LQI': str(self.LQI),
            'ID64': str(self.ID64),
            'CRCPass': str(self.CRCPass),
            'SyncWord': str(self.SyncWord),
            'AntenaID': str(self.AntenaID),
            'IsACK': str(self.IsACK),
            'TimeUs': str(self.Time),
            'RSSI': str(self.RSSI) if self.RSSI != 0 else 'N/A',
            'Channel': str(self.Channel) if self.Channel != 0 else 'N/A',
            'Length': str(self.GetDataLength()),
            'Payload': self.DataBytes,
            'Event': self.Event
        }

@dataclass
class Node:
    Port: str
    Baudrate: int
    TimeOut: float
    Serial: Optional[serial.Serial] = None
    Thread: Optional[threading.Thread] = None
    Data: List[DataEntry] = field(default_factory=list)
    EScan: dict = field(default_factory=dict)
    Packets: List[ZigbeePacket] = field(default_factory=list)
    Status:str = "UNKNOWN"
    BitLength:int = 0
    SyncWords1:int = 0
    SyncWords2:int = 0
    LedSTT:str = "OFF"
    Index: int = 0
    BerRSSI:int = 0
    BitsTested:int = 0
    PercentDone:float = 0
    BitErrors:int = 0
    PercentBitError:float = 0.0
    ID64: str = ""
    def ReadFromUART(Self, CallBack = None):
        Buffer = bytearray()
        while True:
            try:
                Data = Self.Serial.read(Self.Serial.in_waiting or 1)
                if Data:
                    Buffer.extend(Data)
                    while b'\n' in Buffer:
                        Line, _, Buffer = Buffer.partition(b'\n')
                        if len(Line) > 0:
                            if Self.IsPackagePerTest(Line.decode().strip()) == True:
                                if Debug >= 1:
                                    print(f"{Self.Port} Data: {Line.decode().strip()}")
                                if len(Self.Data[Self.Index - 1].TimeRead) == 0:
                                    Self.Data[Self.Index - 1].TimeRead = datetime.now().strftime("%H:%M:%S.%f")[:-3]
                                Self.Data.append(DataEntry(TimeSend=datetime.now().strftime("%H:%M:%S.%f")[:-3], TimeRead="", Cmd='rxPacket', Responses=[]))
                                Self.Index = len(Self.Data)
                                Self.Data[Self.Index - 1].Responses.append(Line.decode(errors='ignore').strip())
                            elif Self.IsGetSyncWords(Line.decode().strip()) == True:
                                if Debug >= 1:
                                    print(f"{Self.Port} Data Sync Words: {Line.decode().strip()}")
                                Pattern = r'(\w+):([^}]+)'
                                Matches = re.findall(Pattern, Line.decode().strip())
                                Result = {}
                                for Key, Value in Matches:
                                    Value = Value.strip()
                                    if Value.isdigit():
                                        Result[Key] = int(Value)
                                    elif Value.replace('.', '').replace('-', '').isdigit():
                                        Result[Key] = float(Value) if '.' in Value else int(Value)
                                    elif Value.lower() in ['true', 'false']:
                                        Result[Key] = Value.lower() == 'true'
                                    else:
                                        Result[Key] = Value
                                if Result.get('Result') == 'Success':
                                    Self.BitLength = int(Result.get('bitlength', '0'))
                                    Self.SyncWords1 = int(Result.get('syncWord1', '0'))
                                    Self.SyncWords2 = int(Result.get('syncWord2', '0'))
                                if Debug >= 2:
                                    print(f"{Self.Port} Node BitLength: {Self.BitLength} SyncWords1: {Self.SyncWords1} SyncWords2: {Self.SyncWords2}")
                            elif Self.IsEScan(Line.decode().strip()) == True:
                                Pattern = r'\{EScan\s+(Ch\d+)\s*:\s*(-?\d+)\}'
                                Match = re.search(Pattern, Line.decode().strip())
                                if Match:
                                    Self.EScan[Match.group(1)] = Match.group(2)
                                if Debug >= 1:
                                    print(f"{Self.Port} EScan Read: {Line.decode().strip()}")
                            elif Self.IsEvent(Line.decode().strip()) == True:
                                NewPacket = ZigbeePacket()
                                Success = NewPacket.ExtractData(Line.decode().strip())
                                if Success:
                                    NewPacket.Decode(NewPacket.DataBytes)
                                    if Debug >= 4:
                                        print(NewPacket.ToPacketTableFormat())
                                        NewPacket.PrintPackage()
                                    Self.Packets.append(NewPacket)
                                if Debug >= 1:
                                    print(f"{Self.Port} Event Read: {Line.decode().strip()}")
                            elif Self.IsBerTest(Line.decode().strip()) == True:
                                Result = Self.ParseBerstatus(Line.decode().strip())
                                Self.BerRSSI = Result['RSSI']
                                Self.BitErrors = Result['BitErrors']
                                Self.PercentDone = Result['PercentDone']
                                Self.BitsTested = Result['BitsTested']
                                Self.PercentBitError = Result['PercentBitError']
                                if Debug >= 1:
                                    print(f"{Self.Port} Ber Test: {Line.decode().strip()}")
                            elif Self.IsLedStt(Line.decode().strip()) == True:
                                Pattern = r'\{LedStt:\s*(\w+)\}'
                                Matches = re.findall(Pattern, Line.decode().strip())
                                if Matches:
                                    Self.LedSTT = Match.group(1)
                                else:
                                    pattern = r'Toggle on/off from (\d{2}) to (\d{2})'
                                    match = re.search(pattern, Line.decode().strip())
                                    if match.group(1) == '01' and match.group(2) == '00':
                                        Self.LedSTT = "OFF"
                                        if Debug >= 1:
                                                print("LED OFF")
                                    if match.group(1) == '00' and match.group(2) == '01':
                                        Self.LedSTT = "ON"
                                        if Debug >= 1:
                                            print("LED ON")
                                if Debug >= 1:
                                        print(f"{Self.Port} Led Read: {Line.decode().strip()}")
                            else:
                                if Debug >= 1:
                                    print(f"{Self.Port} Data Read: {Line.decode().strip()}")
                                if Debug >= 1:
                                    if len(Self.Data[Self.Index - 1].TimeRead) == 0:
                                        Self.Data[Self.Index - 1].TimeRead = datetime.now().strftime("%H:%M:%S.%f")[:-3]
                                    Self.Data[Self.Index - 1].Responses.append(Line.decode(errors='ignore').strip())
            except Exception as Err:
                if Debug >= 4:
                    print(f"{Self.Port} Error: {Err}")
                time.sleep(0.2)
    def Init(Self, GetSW: bool = False):
        """Khởi tạo node với error handling"""
        try:
            if Debug >= 1:
                print(f"{Self.Port} Init Port")
            
            # Đóng connection cũ nếu có
            if Self.Serial and Self.Serial.is_open:
                Self.Serial.close()
                time.sleep(0.2)
            
            # Mở serial connection mới
            Self.Serial = serial.Serial(Self.Port, Self.Baudrate, timeout=Self.TimeOut)
            
            # Khởi tạo thread đọc dữ liệu
            Self.Thread = threading.Thread(target=Self.ReadFromUART, daemon=True)
            Self.Thread.start()
            
            # Khởi tạo EScan dictionary
            Self.EScan = {f"Ch{ch}": float('inf') for ch in range(11, 27)}
            
            # Lấy sync words nếu cần
            if GetSW:
                time.sleep(1)
                Self.SendData("getSyncWords")
                time.sleep(1)
                
            print(f"✅ Node {Self.Port} initialized successfully")
            return True
            
        except Exception as e:
            print(f"❌ Error initializing node {Self.Port}: {e}")
            return False

    def IsEScan(Self, String):
        pattern = r"EScan"
        if re.search(pattern, String):
            escan_pattern = r'\{EScan\s+(Ch\d+)\s*:\s*(-?\d+)\}'
            match = re.search(escan_pattern, String)
            if match:
               channel_str = match.group(1)  # "Ch20"
               energy_value = float(match.group(2))
            
            # Lấy số channel (bỏ "Ch")
               channel_num = int(channel_str.replace('Ch', ''))
               node_id64 = "UNKNOWN"
               if Self.Packets and len(Self.Packets) > 0:
               # Lấy ID64 từ packet gần nhất
                  latest_packet = Self.Packets[-1]
                  if hasattr(latest_packet, 'ID64') and latest_packet.ID64:
                     node_id64 = latest_packet.ID64
                  else:
                   # Fallback: tạo ID64 từ port
                     node_id64 = f"NODE_{Self.Port.replace('COM', '').replace('/', '_')}"
               else:
               # Fallback: tạo ID64 từ port
                     node_id64 = f"NODE_{Self.Port.replace('COM', '').replace('/', '_')}"
               try:
                   save_escan_result(node_id64, channel_num, energy_value)
               except Exception as e: 
                   print(f"❌ EScan save error: {e}")    
            
            
            # Cập nhật EScan dict như cũ
               Self.EScan[channel_str] = str(energy_value)
            return True
        return False
    def ParseBerstatus(Self, String):
        parts = String.strip('{}').split('}{')
        result = {}
        for part in parts:
            if ':' in part:
                key, value = part.split(':', 1)
                try:
                    if '.' in value:
                        value = float(value)
                    else:
                        value = int(value)
                except ValueError:
                    pass
                result[key] = value
        return result
    def IsBerTest(Self, String):
        pattern = r"berstatus"
        if re.search(pattern, String):
            pattern = r"BitsToTest"
            if re.search(pattern, String):
                return True
        return False
    def IsEvent(Self, String):
        pattern = r"Event"
        if re.search(pattern, String):
            return True
        return False
    def IsLedStt(Self, String):
        pattern = r"LedStt"
        if re.search(pattern, String):
            return True
        pattern = r'Toggle on/off from (\d{2}) to (\d{2})'
        match = re.search(pattern, String)
        if match:
            return True
        return False
    def GetLedSTT(Self):
        return Self.LedSTT
    def GetJsonEScan(Self):
        JsonDict = {"Data": "EScanJson"}
        JsonDict.update(Self.EScan)
        JsonStr = json.dumps(JsonDict)
        return JsonStr
    def GetAllPackAge(Self):
        return Self.Packets
    def IsPackagePerTest(Self, String):
        pattern = r"rxPacket"
        if re.search(pattern, String):
            return True
        return False
    def IsGetSyncWords(Self, String):
        pattern = r"getSyncWords"
        if re.search(pattern, String):
            return True
        return False
    def DetectRxPacketDetailed(Self, String):
        Pattern = r"{{(rxPacket)}"
        if not re.search(Pattern, String):
            return False
        RequiredParams = ['len', 'timeUs', 'timePos', 'durationUs', 'crc', 'rssi', 'lqi', 'phy', 'isAck', 'channel', 'payload']
        PatternParams = r"\{(\w+):([^}]+)\}"
        Matches = re.findall(PatternParams, String)
        FoundParams = [Key for Key, value in Matches]
        for Param in RequiredParams:
            if Param not in FoundParams:
                return False
        return True
    def ClosePort(Self):
        if Self.Serial and Self.Serial.is_open:
            Self.Serial.close()
    def SendData(Self, Data, Delay = 0):
        if Debug == 1:
            print(f"{Self.Port} Send data :{Data}")
        Self.Serial.write(f"{Data}\n".encode())
        if Debug >= 1:
            Self.Data.append(DataEntry(TimeSend=datetime.now().strftime("%H:%M:%S.%f")[:-3], TimeRead="", Cmd=Data, Responses=[]))
            Self.Index = len(Self.Data)
        time.sleep(Delay)
    def LogFullData(Self):
        for Command in Self.Data:
            print(f"Send : {Command.TimeSend} {Command.Cmd}")
            print(f"Read : {Command.TimeRead} {Command.Responses}")
    def ExtractBerTest(Self, String):
        Pattern = r"\{(\w+):([^}]+)\}"
        Matches = re.findall(Pattern, String)
        Result = {Key: Value for Key, Value in Matches}
        return Result
    def GetParasBerTest(Self):
        Result = []
        for Cmd in Self.Data:
            if Cmd.Cmd == 'berstatus' and len(Cmd.Responses) > 1:
                Result.append(Self.ExtractBerTest(Cmd.Responses[1]))
        return Result
    def GetPercentDoneBerTest(Self):
        return Self.ExtractBerTest(Self.Data[Self.Index - 1].Responses[1])['PercentDone']
    def ParseRxPacket(Seft, String):
        Pattern = r"rxPacket"
        if not re.search(Pattern, String):
            return None 
        Pattern = r"\{(\w+):([^}]+)\}"
        matches = re.findall(Pattern, String)
        Result = {Key: Value.strip() for Key, Value in matches}
        for Key in ['len', 'timeUs', 'timePos', 'durationUs', 'rssi', 'lqi', 'phy', 'syncWordId', 'antenna', 'channelHopIdx', 'channel']:
            if Key in Result:
                try:
                    Result[Key] = int(float(Result[Key]))
                except ValueError:
                    pass
        if 'isAck' in Result:
            Result['isAck'] = True if Result['isAck'].lower() == 'true' else False 
        if 'payload' in Result:
            PayloadStr = Result['payload'] 
            BytesStr = PayloadStr.replace('0x', '').split()
            try:
                Result['payload'] = [int(Byte, 16) for Byte in BytesStr]
            except ValueError:
                pass
        return Result
    def EScanStart(Self):
        if Debug >= 1:
            print(f"{Self.Port} EScan")
        Self.SendData("escan 0")
    def TurnOnLed(Self):
        if Debug >= 1:
            print(f"{Self.Port} TurnOnLed")
        Self.SendData("zcl on-off on",0.5)
        Self.SendData("send 0xffff 1 1",0.5)
    def TurnOffLed(Self):
        if Debug >= 1:
            print(f"{Self.Port} TurnOffLed")
        Self.SendData("zcl on-off off",0.5)
        Self.SendData("send 0xffff 1 1",0.5)
    def Reset(Self):
        if Debug >= 1:
            print(f"{Self.Port} Reset")
        Self.SendData("reset")
    def LeaveNetwork(Self):
        if Debug >= 1:
            print(f"{Self.Port} Leave")
        Self.SendData("network leave")
    def JoinNetwork(Self):
        if Debug >= 1:
            print(f"{Self.Port} Joind")
        Self.SendData("plugin network-steering start 0")
    def CreateNetwork(Self):
        if Debug >= 1:
            print(f"{Self.Port} Create Network")
        Self.SendData("plugin network-creator form 1 0xABCD 10 15")
    def OpenNetwork(Self):
        if Debug >= 1:
            print(f"{Self.Port} Open Network")
        Self.SendData("plugin network-creator-security open-network")
    def ConfigSyncWords(Self, SyncWords1:int, SyncWords2:int, BitLenght:int):
        Self.SendData(f"reset", 1)
        Self.SendData(f"rx 0", 1)
        Self.SendData(f"configSyncWords {BitLenght} {SyncWords1} {SyncWords2}", 1)
        Self.SendData(f"getSyncWords", 1)
        Self.SendData(f"rx 1", 1)
def GetPorts():
    return [Port.device for Port in list_ports.comports()]
def GetPortsDetailed():
    ports = list_ports.comports()
    PortInfo = []
    for port in ports:
        info = {
            "Device": port.device,
            "Name": port.name,
            "Description": port.description,
            "Hardware ID": port.hwid,
            "Vendor ID": port.vid,
            "Product ID": port.pid,
            "Serial Number": port.serial_number,
            "Location": port.location,
            "Manufacturer": port.manufacturer,
            "Product": port.product,
            "Interface": port.interface
        }
        PortInfo.append(info)
    return PortInfo
def BerTest(NodeSend: Node, NodeRead: Node, NumBits: int = 2000000, TimeOut:int = 1000, Reset:bool = True, DebugMode:bool = True, Freq:int = 2405000000):
    # if Reset == True: 
    if DebugMode == True:
        # if Reset == False:
            # NodeSend.SendData("reset")
            # time.sleep(0.5)
            # NodeRead.SendData("reset")
            # time.sleep(0.5)
        NodeSend.SendData(f"rx {0}")
        time.sleep(0.5)
        NodeRead.SendData(f"rx {0}")
        time.sleep(0.5)
        NodeSend.SendData(f"setDebugMode {1}")
        time.sleep(0.5)
        NodeRead.SendData(f"setDebugMode {1}")
        time.sleep(0.5)
        if DebugMode == True:
            NodeSend.SendData(f"freqOverride {Freq}")
            time.sleep(0.5)
            NodeRead.SendData(f"freqOverride {Freq}")
            time.sleep(0.5)
        NodeRead.SendData(f"setberconfig {NumBits}")
        time.sleep(0.5)
        NodeRead.SendData(f"rx {1}")
        time.sleep(0.5)
        NodeRead.SendData("berrx 1")
        time.sleep(0.1)

        NodeSend.SendData("SetTxStream 1 1")
        time.sleep(0.5)
    for I in range(TimeOut):
        NodeRead.SendData("getrssi")
        time.sleep(0.5)
        NodeRead.SendData("berstatus")
        time.sleep(0.5)
        # Done = float(NodeRead.GetPercentDoneBerTest())
        Done = NodeRead.PercentDone
        if Debug == 0:
            print(f"\rPercent Done: {Done:.2f}% BTD: {NodeRead.BitsTested} RSSI: {NodeRead.BerRSSI} BER: {NodeRead.BitErrors}", end="")
        if Debug >= 1:
            print(f"\rPercent Done: {Done:.2f}% BTD: {NodeRead.BitsTested} RSSI: {NodeRead.BerRSSI} BER: {NodeRead.BitErrors}")
        sys.stdout.flush()
        if Done == 100.00:
            NodeSend.SendData("reset")
            time.sleep(0.5)
            NodeRead.SendData("reset")
            time.sleep(0.5)
            break


In [4]:
from flask import Flask, render_template, request, jsonify, send_from_directory
import json
import os
import threading
import time
from datetime import datetime
import sqlite3
import re
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field

# Import các class từ paste.txt
# from paste import Node, ZigbeePacket, GetPorts, save_escan_result

app = Flask(__name__)
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)
# Global variables
nodes: Dict[str, Node] = {}
ber_test_nodes: Dict[str, Node] = {}

def create_database():
    """Tạo database cho zigbee packets"""
    conn = sqlite3.connect('zigbee_packets.db')
    cursor = conn.cursor()
    
    
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS zigbee_packets (
            ID TEXT,
            event TEXT,
            data TEXT,
            time INTEGER,
            rssi INTEGER,
            antena_id INTEGER,
            sync_word INTEGER,
            is_ack INTEGER,
            crc_pass INTEGER,
            lqi INTEGER,
            channel INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    print("✅ Database zigbee_packets.db đã được tạo thành công")

def create_escan_database():
    """Tạo database cho EScan với ID64"""
    conn = sqlite3.connect('escan_results.db')
    cursor = conn.cursor()
    
   
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS escan_data (
            ID64 TEXT NOT NULL,
            channel INTEGER NOT NULL,
            energy_value REAL NOT NULL,
            scan_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (ID64, channel, scan_timestamp)
        )
    ''')
    
    conn.commit()
    conn.close()
    print("✅ Database escan_results.db đã được tạo với ID64")

@app.route('/')
def index():
    """Trang chính"""
    return render_template('index.html')

@app.route('/static/<path:filename>')
def static_files(filename):
    """Serve static files"""
    return send_from_directory('static', filename)

@app.route('/refresh_ports', methods=['GET'])
def refresh_ports():
    """Refresh danh sách COM ports"""
    try:
        ports = GetPorts()
        return jsonify(ports)
    except Exception as e:
        print(f"❌ Error refreshing ports: {e}")
        return jsonify([])

@app.route('/open_port', methods=['POST'])
def open_port():
    """Mở port cho node"""
    try:
        data = request.get_json()
        node_id = data.get('node_id')
        port = data.get('port')
        baudrate = data.get('baudrate', 115200)
        
        # KIỂM TRA VÀ XỬ LÝ NODE ĐÃ TỒN TẠI
        if node_id in nodes:
            print(f"⚠️ Node {node_id} already exists, cleaning up first...")
            try:
                existing_node = nodes[node_id]
                # Đóng serial connection cũ
                if existing_node.Serial and existing_node.Serial.is_open:
                    existing_node.Serial.close()
                    print(f"✅ Closed existing connection for {node_id}")
                
                # Xóa node cũ khỏi dictionary
                del nodes[node_id]
                print(f"🗑️ Removed existing node {node_id}")
                
                # Delay ngắn để đảm bảo port được giải phóng
                time.sleep(0.5)
                
            except Exception as cleanup_error:
                print(f"⚠️ Cleanup error for {node_id}: {cleanup_error}")
                # Tiếp tục tạo node mới dù cleanup có lỗi
        
        # Tạo node mới
        new_node = Node(
            Port=port,
            Baudrate=baudrate,
            TimeOut=1.0
        )
        
        # Khởi tạo node
        success = new_node.Init(GetSW=True)
        
        if success:
            nodes[node_id] = new_node
            
            return jsonify({
                'status': 'success',
                'message': f'Đã mở port {port} cho {node_id}'
            })
        else:
            return jsonify({
                'status': 'error',
                'message': f'Không thể khởi tạo node trên {port}'
            })
        
    except Exception as e:
        print(f"❌ Error opening port: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi mở port: {str(e)}'
        })

@app.route('/close_port', methods=['POST'])
def close_port():
    """Đóng port cho node"""
    try:
        data = request.get_json()
        node_id = data.get('node_id')
        
        if node_id in nodes:
            nodes[node_id].ClosePort()
            del nodes[node_id]
            
        return jsonify({
            'status': 'success',
            'message': f'Đã đóng port cho {node_id}'
        })
        
    except Exception as e:
        print(f"❌ Error closing port: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi đóng port: {str(e)}'
        })

@app.route('/remove_device', methods=['POST'])
def remove_device():
    """Xóa device"""
    try:
        data = request.get_json()
        node_id = data.get('node_id')
        
        if node_id in nodes:
            nodes[node_id].ClosePort()
            del nodes[node_id]
            
        return jsonify({
            'status': 'success',
            'message': f'Đã xóa device {node_id}'
        })
        
    except Exception as e:
        print(f"❌ Error removing device: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi xóa device: {str(e)}'
        })

@app.route('/zigbee_action', methods=['POST'])
def zigbee_action():
    """Thực hiện các hành động ZigBee"""
    try:
        data = request.get_json()
        node_id = data.get('node_id')
        action = data.get('action')
        
        if node_id not in nodes:
            return jsonify({
                'status': 'error',
                'message': f'Node {node_id} không tồn tại'
            })
        
        node = nodes[node_id]
        
        # Thực hiện action
        if action == 'reset':
            node.Reset()
        elif action == 'create_network':
            node.CreateNetwork()
        elif action == 'open_network':
            node.OpenNetwork()
        elif action == 'leave_network':
            node.LeaveNetwork()
        elif action == 'join_network':
            node.JoinNetwork()
        elif action == 'toggle_led':
            # Coordinator toggle LED
            if node.LedSTT == "ON":
                node.TurnOffLed()
            else:
                node.TurnOnLed()
        elif action == 'turn_on_led':
            node.TurnOnLed()
        elif action == 'turn_off_led':
            node.TurnOffLed()
        else:
            return jsonify({
                'status': 'error',
                'message': f'Action không hợp lệ: {action}'
            })
        
        # Trả về trạng thái LED nếu có
        response = {
            'status': 'success',
            'message': f'Đã thực hiện {action}'
        }
        
        if action in ['toggle_led', 'turn_on_led', 'turn_off_led']:
            response['led_status'] = node.LedSTT
            
        return jsonify(response)
        
    except Exception as e:
        print(f"❌ Error in zigbee_action: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi thực hiện action: {str(e)}'
        })

@app.route('/get_led_state/<node_id>', methods=['GET'])
def get_led_state(node_id):
    """Lấy trạng thái LED của node"""
    try:
        if node_id not in nodes:
            return jsonify({
                'status': 'error',
                'message': f'Node {node_id} không tồn tại'
            })
        
        return jsonify({
            'status': 'success',
            'led_state': nodes[node_id].LedSTT
        })
        
    except Exception as e:
        print(f"❌ Error getting LED state: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi lấy trạng thái LED: {str(e)}'
        })

@app.route('/start_escan/<node_id>', methods=['POST'])
def start_escan(node_id):
    """Bắt đầu Energy Scan"""
    try:
        if node_id not in nodes:
            return jsonify({
                'status': 'error',
                'message': f'Node {node_id} không tồn tại'
            })
        
        nodes[node_id].EScanStart()
        
        return jsonify({
            'status': 'success',
            'message': 'Energy scan đã bắt đầu'
        })
        
    except Exception as e:
        print(f"❌ Error starting EScan: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi bắt đầu EScan: {str(e)}'
        })

@app.route('/get_escan_data/<node_id>', methods=['GET'])
def get_escan_data(node_id):
    """Lấy dữ liệu Energy Scan"""
    try:
        if node_id not in nodes:
            return jsonify({
                'status': 'error',
                'message': f'Node {node_id} không tồn tại'
            })
        
        escan_data = nodes[node_id].EScan
        
        return jsonify({
            'status': 'success',
            'data': json.dumps(escan_data)
        })
        
    except Exception as e:
        print(f"❌ Error getting EScan data: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi lấy dữ liệu EScan: {str(e)}'
        })

@app.route('/get_all_packages/<node_id>', methods=['GET'])
def get_all_packages(node_id):
    """Lấy tất cả packages của node"""
    try:
        if node_id not in nodes:
            return jsonify([])
        
        node = nodes[node_id]
        packages = []
        
        for packet in node.Packets:
            package_data = packet.ToPacketTableFormat()
            # Thêm TypePackage vào response
            package_data['TypePackage'] = packet.TypePackage
            # Tạo real time cho mỗi packet
            package_data['RealTime'] = datetime.now().strftime("%H:%M:%S.%f")[:-3]
            packages.append(package_data)
        
        return jsonify(packages)
        
    except Exception as e:
        print(f"❌ Error getting packages: {e}")
        return jsonify([])

@app.route('/get_packet_detail/<node_id>', methods=['POST'])
def get_packet_detail(node_id):
    """Lấy chi tiết packet để hiển thị trong modal theo phong cách Wireshark"""
    try:
        if node_id not in nodes:
            return jsonify({
                'status': 'error',
                'message': f'Node {node_id} không tồn tại'
            })
        
        data = request.get_json()
        packet_data = data.get('packet')
        
        # Tìm packet tương ứng trong node
        node = nodes[node_id]
        target_packet = None
        
        for packet in node.Packets:
            if (packet.Event == packet_data.get('Event') and 
                packet.Time == int(packet_data.get('TimeUs', 0))):
                target_packet = packet
                break
        
        if not target_packet:
            # Tạo packet từ dữ liệu được gửi
            target_packet = create_packet_from_data(packet_data)
        
        # Tạo chi tiết packet theo phong cách Wireshark
        detail = create_wireshark_packet_detail(target_packet)
        
        return jsonify({
            'status': 'success',
            'detail': detail
        })
        
    except Exception as e:
        print(f"❌ Error getting packet detail: {e}")
        return jsonify({
            'status': 'error',
            'message': f'Lỗi khi lấy chi tiết packet: {str(e)}'
        })

def create_packet_from_data(packet_data):
    """Tạo ZigbeePacket từ dữ liệu packet"""
    packet = ZigbeePacket()
    packet.Event = packet_data.get('Event', '')
    packet.Time = int(packet_data.get('TimeUs', 0))
    packet.ID64 = packet_data.get('ID64', '')
    packet.RSSI = int(packet_data.get('RSSI', 0)) if packet_data.get('RSSI') != 'N/A' else 0
    packet.Channel = int(packet_data.get('Channel', 0)) if packet_data.get('Channel') != 'N/A' else 0
    packet.LQI = int(packet_data.get('LQI', 0))
    packet.CRCPass = int(packet_data.get('CRCPass', 0))
    packet.SyncWord = int(packet_data.get('SyncWord', 0))
    packet.AntenaID = int(packet_data.get('AntenaID', 0))
    packet.IsACK = int(packet_data.get('IsACK', 0))
    packet.TypePackage = packet_data.get('TypePackage', 'Unknown')
    
    # Xử lý payload
    payload = packet_data.get('Payload', [])
    if isinstance(payload, list):
        packet.DataBytes = [str(b) for b in payload]
        packet.ApplicationPayload = packet.DataBytes.copy()
    
    return packet

def create_wireshark_packet_detail(packet: ZigbeePacket):
    """Tạo chi tiết packet theo phong cách Wireshark"""
    detail = {
        'TypePackage': packet.TypePackage,
        'LQI': packet.LQI,
        'CRCPass': packet.CRCPass,
        'SyncWord': packet.SyncWord,
        'AntenaID': packet.AntenaID,
        'DataBytes': packet.DataBytes,
        'ApplicationPayload': packet.ApplicationPayload
    }
    
    # Thêm thông tin IEEE 802.15.4 với chi tiết đầy đủ
    if packet.Interface:
        detail['Interface'] = {
            'PHYHeader': packet.Interface.PHYHeader,
            'Sequence': packet.Interface.Sequence,
            'DestinationPANID': packet.Interface.DestinationPANID,
            'ShortDestinationAddr': packet.Interface.ShortDestinationAddr,
            'ShortSouceAddr': packet.Interface.ShortSouceAddr,
            'FrameControl': {
                'FrameControl': packet.Interface.FrameControl.FrameControl,
                'FrameType': packet.Interface.FrameControl.FrameType,
                'SecurityEn': packet.Interface.FrameControl.SecurityEn,
                'FramePending': packet.Interface.FrameControl.FramePending,
                'AckRequied': packet.Interface.FrameControl.AckRequied,
                'PanIDCompression': packet.Interface.FrameControl.PanIDCompression,
                'FrameVersion': packet.Interface.FrameControl.FrameVersion,
                'Reserved': packet.Interface.FrameControl.Reserved,
                'DestinationAddrMode': packet.Interface.FrameControl.DestinationAddrMode,
                'SouceAddrMode': packet.Interface.FrameControl.SouceAddrMode
            }
        }
    
    # Thêm thông tin ZigBee Network với chi tiết đầy đủ
    if packet.ZigbeeNet and packet.ZigbeeNet.FrameControl.FrameControl:
        detail['ZigbeeNet'] = {
            'DestinationAddr': packet.ZigbeeNet.DestinationAddr,
            'SouceAddr': packet.ZigbeeNet.SouceAddr,
            'Radius': packet.ZigbeeNet.Radius,
            'Sequence': packet.ZigbeeNet.Sequence,
            'FrameControl': {
                'FrameControl': packet.ZigbeeNet.FrameControl.FrameControl,
                'FrameType': packet.ZigbeeNet.FrameControl.FrameType,
                'ProtocoVersion': packet.ZigbeeNet.FrameControl.ProtocoVersion,
                'DiscoverRouter': packet.ZigbeeNet.FrameControl.DiscoverRouter,
                'Multicast': packet.ZigbeeNet.FrameControl.Multicast,
                'Security': packet.ZigbeeNet.FrameControl.Security,
                'SouceRouter': packet.ZigbeeNet.FrameControl.SouceRouter,
                'LongDestPresent': packet.ZigbeeNet.FrameControl.LongDestPresent,
                'LongSoucePresent': packet.ZigbeeNet.FrameControl.LongSoucePresent,
                'EndDeviceInitiator': packet.ZigbeeNet.FrameControl.EndDeviceInitiator
            }
        }
    
    # Thêm thông tin ZigBee Security với chi tiết đầy đủ
    if packet.ZigbeeNetSec and packet.ZigbeeNetSec.FrameControl.FrameControl:
        detail['ZigbeeNetSec'] = {
            'FrameCouter': packet.ZigbeeNetSec.FrameCouter,
            'SouceAddr': packet.ZigbeeNetSec.SouceAddr,
            'KeySequence': packet.ZigbeeNetSec.KeySequence,
            'FrameControl': {
                'FrameControl': packet.ZigbeeNetSec.FrameControl.FrameControl,
                'SecurityLevel': packet.ZigbeeNetSec.FrameControl.SecurityLevel,
                'KeyIdentifer': packet.ZigbeeNetSec.FrameControl.KeyIdentifer,
                'ExtenedNonce': packet.ZigbeeNetSec.FrameControl.ExtenedNonce
            }
        }
    
    return detail

# ===== BERTEST ROUTES - SỬA LẠI =====
# BerTest global variables
bertest_nodes = {}
bertest_running = False
bertest_thread = None
bertest_progress = 0
bertest_tx_node = None
bertest_rx_node = None
# THÊM: Lưu kết quả cuối cùng
bertest_final_results = {
    "progress": 0.0,
    "ber_rssi": 0,
    "bit_errors": 0,
    "bits_tested": 0,
    "percent_bit_error": 0.0,
    "completed": False
}

@app.route('/bertest_page')
def bertest_page():
    """Render BerTest page"""
    return render_template('bertest.html')

@app.route('/bertest_open_port', methods=['POST'])
def bertest_open_port():
    """Mở port cho BerTest node"""
    try:
        data = request.json
        node_id = data['node_id']
        port = data['port']
        baudrate = int(data.get('baudrate', 115200))
        
        # Đóng node cũ nếu có
        if node_id in bertest_nodes:
            try:
                bertest_nodes[node_id].ClosePort()
            except:
                pass
            del bertest_nodes[node_id]
        
        # Tạo node mới với classes từ paste.txt
        node = Node(Port=port, Baudrate=baudrate, TimeOut=0.2)
        node.Init(GetSW=True)  # Lấy sync words mặc định
        bertest_nodes[node_id] = node
        
        print(f"✅ BerTest: Opened port {port} for {node_id}")
        return jsonify({
            "status": "success", 
            "message": f"Connected {node_id} to {port}",
            "sync_words1": node.SyncWords1,
            "sync_words2": node.SyncWords2,
            "bit_length": node.BitLength
        })
    except Exception as e:
        return jsonify({
            "status": "error", 
            "message": f"Port opening error: {str(e)}"
        })

@app.route('/bertest_close_port', methods=['POST'])
def bertest_close_port():
    """Đóng port cho BerTest node"""
    try:
        data = request.json
        node_id = data['node_id']
        
        if node_id in bertest_nodes:
            try:
                bertest_nodes[node_id].ClosePort()
            except Exception as e:
                print(f"Error closing BerTest port: {e}")
            del bertest_nodes[node_id]
            return jsonify({
                "status": "success", 
                "message": f"Disconnected {node_id}"
            })
        return jsonify({
            "status": "error", 
            "message": "Node does not exist"
        })
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)})

@app.route('/bertest_config_sync', methods=['POST'])
def bertest_config_sync():
    """Cấu hình Sync Words cho node"""
    try:
        data = request.json
        node_id = data['node_id']
        sync_words1 = int(data['sync_words1'])
        sync_words2 = int(data['sync_words2'])
        bit_length = int(data['bit_length'])
        
        if node_id not in bertest_nodes:
            return jsonify({
                "status": "error", 
                "message": "Node not connected"
            })
        
        node = bertest_nodes[node_id]
        # Sử dụng method ConfigSyncWords từ paste.txt
        node.ConfigSyncWords(sync_words1, sync_words2, bit_length)
        
        # Đợi một chút để node cập nhật
        time.sleep(0.5)
        
        return jsonify({
            "status": "success", 
            "message": f"Sync words configured for {node_id}",
            "sync_words1": node.SyncWords1,
            "sync_words2": node.SyncWords2,
            "bit_length": node.BitLength
        })
    except Exception as e:
        return jsonify({
            "status": "error", 
            "message": str(e)
        })

@app.route('/bertest_start', methods=['POST'])
def bertest_start():
    """Bắt đầu BER Test"""
    global bertest_running, bertest_thread, bertest_progress, bertest_tx_node, bertest_rx_node, bertest_final_results
    
    try:
        data = request.json
        tx_node_id = data['tx_node']
        rx_node_id = data['rx_node']
        num_bits = int(data.get('num_bits', 1000000))
        freq = int(data.get('freq', 2405000000))
        timeout = int(data.get('timeout', 1000))
        
        # Kiểm tra nodes tồn tại
        if tx_node_id not in bertest_nodes or rx_node_id not in bertest_nodes:
            return jsonify({
                "status": "error",
                "message": "Both nodes must be connected"
            })
        
        if bertest_running:
            return jsonify({
                "status": "error",
                "message": "BER Test is already running"
            })
        
        # Reset kết quả trước khi bắt đầu test mới
        bertest_final_results = {
            "progress": 0.0,
            "ber_rssi": 0,
            "bit_errors": 0,
            "bits_tested": 0,
            "percent_bit_error": 0.0,
            "completed": False
        }
        
        # Lưu thông tin TX/RX nodes
        bertest_tx_node = tx_node_id
        bertest_rx_node = rx_node_id
        
        # Lấy nodes
        node_send = bertest_nodes[tx_node_id]
        node_read = bertest_nodes[rx_node_id]
        
        # Reset progress và thông số BER
        bertest_progress = 0
        bertest_running = True
        
        # Reset các thông số BER của RX node
        node_read.PercentDone = 0.0
        node_read.BitsTested = 0
        node_read.BitErrors = 0
        node_read.PercentBitError = 0.0
        node_read.BerRSSI = 0
        
        # Chạy BerTest trong thread riêng
        def run_bertest():
            global bertest_running, bertest_progress, bertest_final_results
            try:
                print(f"🔬 Starting BER Test: {tx_node_id}(TX) -> {rx_node_id}(RX)")
                print(f"📊 Parameters: {num_bits} bits, {freq} Hz, {timeout}s timeout")
                
                # Gọi hàm BerTest từ paste.txt
                BerTest(
                    NodeSend=node_send,
                    NodeRead=node_read,
                    NumBits=num_bits,
                    TimeOut=timeout,
                    Reset=True,
                    DebugMode=True,
                    Freq=freq
                )
                
                # LƯU KẾT QUẢ CUỐI CÙNG TRƯỚC KHI DỪNG
                if bertest_rx_node and bertest_rx_node in bertest_nodes:
                    rx_node = bertest_nodes[bertest_rx_node]
                    try:
                        bertest_final_results = {
                            "progress": float(getattr(rx_node, 'PercentDone', 100.0)),
                            "ber_rssi": int(getattr(rx_node, 'BerRSSI', 0)),
                            "bit_errors": int(getattr(rx_node, 'BitErrors', 0)),
                            "bits_tested": int(getattr(rx_node, 'BitsTested', 0)),
                            "percent_bit_error": float(getattr(rx_node, 'PercentBitError', 0.0)),
                            "completed": True
                        }
                        print(f"💾 Final Results Saved: {bertest_final_results}")
                    except Exception as e:
                        print(f"❌ Error saving final results: {e}")
                
                print("✅ BER Test completed successfully")
                
            except Exception as e:
                print(f"❌ BER Test error: {str(e)}")
            finally:
                bertest_running = False
        
        bertest_thread = threading.Thread(target=run_bertest, daemon=True)
        bertest_thread.start()
        
        return jsonify({
            "status": "success",
            "message": "BER Test started successfully"
        })
        
    except Exception as e:
        bertest_running = False
        return jsonify({
            "status": "error",
            "message": str(e)
        })

@app.route('/bertest_status')
def bertest_status():
    """Kiểm tra trạng thái BER Test với thông tin chi tiết"""
    global bertest_running, bertest_progress, bertest_rx_node, bertest_final_results
    
    try:
        if bertest_running and bertest_rx_node and bertest_rx_node in bertest_nodes:
            # TEST ĐANG CHẠY: Lấy dữ liệu real-time từ RX node
            rx_node = bertest_nodes[bertest_rx_node]
            try:
                progress_value = float(getattr(rx_node, 'PercentDone', 0.0))
                ber_rssi = int(getattr(rx_node, 'BerRSSI', 0))
                bit_errors = int(getattr(rx_node, 'BitErrors', 0))
                bits_tested = int(getattr(rx_node, 'BitsTested', 0))
                percent_bit_error = float(getattr(rx_node, 'PercentBitError', 0.0))
                
                # Cập nhật kết quả trong quá trình chạy
                bertest_final_results.update({
                    "progress": progress_value,
                    "ber_rssi": ber_rssi,
                    "bit_errors": bit_errors,
                    "bits_tested": bits_tested,
                    "percent_bit_error": percent_bit_error,
                    "completed": False
                })
                
                print(f"🔄 Live Data: {progress_value:.2f}% | {bits_tested} bits | {bit_errors} errors | {ber_rssi} dBm")
                
                return jsonify({
                    "status": "success",
                    "running": True,
                    "progress": progress_value,
                    "ber_rssi": ber_rssi,
                    "bit_errors": bit_errors,
                    "bits_tested": bits_tested,
                    "percent_bit_error": percent_bit_error,
                    "completed": False
                })
                
            except Exception as e:
                print(f"❌ Error reading live data: {e}")
                
        # TEST ĐÃ HOÀN THÀNH: Trả về kết quả cuối cùng đã lưu
        return jsonify({
            "status": "success",
            "running": bertest_running,
            "progress": bertest_final_results["progress"],
            "ber_rssi": bertest_final_results["ber_rssi"],
            "bit_errors": bertest_final_results["bit_errors"],
            "bits_tested": bertest_final_results["bits_tested"],
            "percent_bit_error": bertest_final_results["percent_bit_error"],
            "completed": bertest_final_results["completed"]
        })
        
    except Exception as e:
        return jsonify({
            "status": "error",
            "message": str(e)
        })

@app.route('/bertest_stop', methods=['POST'])
def bertest_stop():
    """Dừng BER Test"""
    global bertest_running, bertest_thread, bertest_tx_node, bertest_rx_node
    
    try:
        bertest_running = False
        bertest_tx_node = None
        bertest_rx_node = None
        
        if bertest_thread and bertest_thread.is_alive():
            # Python không thể force kill thread, nó sẽ tự dừng
            pass
        
        return jsonify({
            "status": "success",
            "message": "BER Test stopped"
        })
    except Exception as e:
        return jsonify({
            "status": "error",
            "message": str(e)
        })

@app.route('/bertest_clear_results', methods=['POST'])
def bertest_clear_results():
    """Xóa kết quả BER test"""
    global bertest_final_results
    
    try:
        bertest_final_results = {
            "progress": 0.0,
            "ber_rssi": 0,
            "bit_errors": 0,
            "bits_tested": 0,
            "percent_bit_error": 0.0,
            "completed": False
        }
        
        return jsonify({
            "status": "success",
            "message": "BER Test results cleared"
        })
    except Exception as e:
        return jsonify({
            "status": "error",
            "message": str(e)
        })
def cleanup_all_nodes():
    """Cleanup tất cả nodes và đóng serial connections"""
    global cleanup_in_progress, nodes
    
    if cleanup_in_progress:
        return
        
    cleanup_in_progress = True
    
    print("🧹 Đang đóng tất cả kết nối...")
    nodes_to_remove = list(nodes.keys()) if 'nodes' in globals() else []
    
    for node_id in nodes_to_remove:
        try:
            if node_id in nodes:
                node = nodes[node_id]
                if hasattr(node, 'Serial') and node.Serial and node.Serial.is_open:
                    node.Serial.close()
                    print(f"✅ Đã đóng kết nối {node_id}")
                    
        except Exception as e:
            print(f"⚠️ Lỗi khi đóng {node_id}: {e}")
    
    # Clear nodes dictionary
    if 'nodes' in globals():
        nodes.clear()
        
    print(f"🧹 Đã dọn dẹp {len(nodes_to_remove)} nodes")
    cleanup_in_progress = False
    return nodes_to_remove

# Global variables
nodes = {}  # Đảm bảo biến global được khai báo
cleanup_in_progress = False
@app.route('/bertest_reset_node', methods=['POST'])
def bertest_reset_node():
    """Reset node - THÊM MỚI"""
    try:
        data = request.json
        node_id = data['node_id']
        
        if node_id not in bertest_nodes:
            return jsonify({
                "status": "error", 
                "message": "Node not connected"
            })
        
        node = bertest_nodes[node_id]
        # Gọi method Reset() từ Node class
        node.Reset()
        
        return jsonify({
            "status": "success", 
            "message": f"Node {node_id} reset successfully"
        })
    except Exception as e:
        return jsonify({
            "status": "error", 
            "message": str(e)
        })
# ===== CLI ROUTES =====
@app.route('/send_cli_command', methods=['POST'])
def send_cli_command():
    """
    Gửi lệnh CLI bằng phương thức Node.SendData, sau đó chờ và lấy
    phản hồi từ danh sách Data do một module khác điền vào.
    """
    try:
        data = request.get_json()
        node_id = data.get('node_id')
        command = data.get('command', '').strip()

        if not node_id or node_id not in nodes:
            return jsonify({'status': 'error', 'message': f'Node {node_id} không hợp lệ hoặc không tồn tại'})

        if not command:
            return jsonify({'status': 'error', 'message': 'Lệnh không được để trống'})

        node = nodes[node_id]

        if not hasattr(node, 'Serial') or not node.Serial or not node.Serial.is_open:
            return jsonify({'status': 'error', 'message': 'Kết nối serial chưa được mở. Vui lòng "Open Port" trước.'})

        # ==================== LOGIC NÂNG CẤP BẮT ĐẦU TỪ ĐÂY ====================

        # 1. Gửi lệnh đi bằng phương thức được yêu cầu: Node.SendData()
        # Giả định phương thức này tồn tại trong class Node của bạn.
        if not hasattr(node, 'SendData'):
             return jsonify({'status': 'error', 'message': 'Lỗi hệ thống: Phương thức SendData không tồn tại trên Node.'})
        
        node.SendData(Data=command, Delay=0)

        # 2. Chờ phản hồi xuất hiện trong danh sách node.Data
        timeout = 10.0  # Chờ tối đa 10 giây
        start_time = time.time()

        while time.time() - start_time < timeout:
            for entry in reversed(node.Data):
                # Điều kiện tìm kiếm:
                # - Lệnh phải khớp (entry.Cmd == command)
                # - Phản hồi phải có nội dung (entry.Responses)
                if entry.Cmd == command and entry.Responses:
                    
                    # 3. Lấy kết quả và định dạng để trả về
                    response_str = '\n'.join(entry.Responses)
                    
                    # 4. (QUAN TRỌNG) Xóa entry đã xử lý khỏi danh sách
                    # để tránh bị lấy lại trong các lần gọi sau (đặc biệt khi gửi cùng một lệnh nhiều lần)
                    try:
                        node.Data.remove(entry)
                    except ValueError:
                        # Entry có thể đã bị xóa bởi một thread khác, điều này không sao
                        pass

                    return jsonify({
                        'status': 'success',
                        'response': response_str,
                        'timestamp': entry.TimeRead  # Sử dụng timestamp chính xác từ DataEntry
                    })
            
            # Tạm dừng một chút trước khi kiểm tra lại để giảm tải CPU
            time.sleep(0.1)

        # 5. Xử lý khi hết thời gian chờ (timeout)
        return jsonify({
            'status': 'error',
            'message': f'Timeout: Không nhận được phản hồi cho lệnh "{command}" trong vòng {int(timeout)} giây.'
        })

    except Exception as e:
        print(f"❌ Lỗi nghiêm trọng khi gửi lệnh CLI: {e}")
        return jsonify({'status': 'error', 'message': f'Lỗi hệ thống khi gửi lệnh: {str(e)}'})



@app.route('/get_cli_history/<node_id>', methods=['GET'])
def get_cli_history(node_id):
    """Lấy lịch sử lệnh CLI (có thể mở rộng sau)"""
    try:
        # Placeholder cho tương lai nếu muốn lưu lịch sử
        return jsonify({
            'status': 'success',
            'history': []
        })
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        })
    
if __name__ == '__main__':
    # Tạo thư mục templates và static nếu chưa có
    os.makedirs('templates', exist_ok=True)
    os.makedirs('static/css', exist_ok=True)
    os.makedirs('static/js', exist_ok=True)
    
    # Tạo databases
    create_database()
    create_escan_database()
    
    print("🚀 Khởi động Zigbee Multi-Node Controller...")
    print("📱 Truy cập: http://localhost:5000")
    print("🔧 Nhấn Ctrl+C để dừng server và đóng tất cả kết nối")
    
    try:
        app.run(debug=False, host='0.0.0.0', port=5000, threaded=True)
    except KeyboardInterrupt:
        print("\n🛑 Đang dừng server...")
        cleanup_all_nodes()
        print("✅ Đã đóng tất cả kết nối")
    except Exception as e:
        print(f"❌ Lỗi server: {e}")
        cleanup_all_nodes()
        print("✅ Đã đóng tất cả kết nối")
    finally:
        # Đảm bảo cleanup trong mọi trường hợp
        cleanup_all_nodes()
        print("🔚 Server đã dừng hoàn toàn")


✅ Database zigbee_packets.db đã được tạo thành công
✅ Database escan_results.db đã được tạo với ID64
🚀 Khởi động Zigbee Multi-Node Controller...
📱 Truy cập: http://localhost:5000
🔧 Nhấn Ctrl+C để dừng server và đóng tất cả kết nối
 * Serving Flask app '__main__'
 * Debug mode: off
🧹 Đang đóng tất cả kết nối...
🧹 Đã dọn dẹp 0 nodes
🔚 Server đã dừng hoàn toàn
